In [1]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg

In [2]:
spatrix=sp.coo_matrix
sp_format='coo'

In [3]:
si=spatrix(np.array([[1,0],[0,1]]),dtype=complex)/2
sz=spatrix(np.array([[1,0],[0,-1]]),dtype=complex)/2
sx=spatrix(np.array([[0,1],[1,0]]),dtype=complex)/2
sy=spatrix(np.array([[0,-1j],[1j,0]]),dtype=complex)/2


In [4]:
def lkron(arr):
    acc=arr[0]
    for i in range(1,len(arr)):
        acc = sp.kron(acc,arr[i],sp_format)
    return acc

In [5]:
def sign(i):
    return +1 if i>0 else -1 if i<0 else 0
def NAE3SAT_2_ham(NAE3SAT):
    size = max([max([abs(lit) for lit in clause]) for clause in NAE3SAT])
    #ham
    first = True
    for cl in NAE3SAT:
        #print(cl)
        s1=[si for k in range(size)]
        s2=[si for k in range(size)]
        s3=[si for k in range(size)]
        s4=[si for k in range(size)]
        s2[abs(cl[0])-1] = sign(cl[0])*sz
        s2[abs(cl[1])-1] = sign(cl[1])*sz
        s3[abs(cl[1])-1] = sign(cl[1])*sz
        s3[abs(cl[2])-1] = sign(cl[2])*sz
        s4[abs(cl[2])-1] = sign(cl[2])*sz
        s4[abs(cl[0])-1] = sign(cl[0])*sz
        #print(s1)
        #print(lkron(s1))
        #print(lkron(s1)+lkron(s2))
        if first:
            ham = (lkron(s1)+lkron(s2)+lkron(s3)+lkron(s4))/4
            first = False
        else:
            ham+= (lkron(s1)+lkron(s2)+lkron(s3)+lkron(s4))/4
    return ham

задаём NAE3SAT (в difficult-SAT.ipynb уже проверили, что там только 2 решения (у NAE3SAT всегда чётное число решений))
и строим hp - гамильтониан проблемы

In [124]:
NAE3SAT = [[1, -2, -6], [-1, 2, 6], [-1, -2, -7], [1, -2, -7], [-1, -3, -4], 
     [1, 3, 5], [1, -3, 5], [-1, 4, -6], [-1, -5, -7], [-2, -3, 4], 
     [2, -4, -6], [-2, 4, -8], [-3, -4, 5], [-3, 5, -8], [3, 6, -7], 
     [-4, -5, -6], [4, 6, 7], [-4, 6, 8], [4, 6, 8], [4, -6, 8], [-5, -6, 8], [-5, 7, -8], [-5, -7, 8]]
size = max([max([abs(lit) for lit in clause]) for clause in NAE3SAT])
hp = NAE3SAT_2_ham(NAE3SAT)
print(size)
#print(hp)
#np.array(hp)


8


ищем СВ hp и их ненулевые координаты дают нам решения (которые совпадают с теми, что выдал pycosat)

In [94]:
e,v = sp.linalg.eigsh(hp,3,which='SA',return_eigenvectors =True)
print(sorted(e))

[2.252914214981671e-19, 8.105595102780888e-19, 0.003906249999999993]


In [95]:
#v2 = [[v[j,i] if abs(v[j,i])>1e-12 else 0 for j in range(len(v))] for i in range(len(v[0]))]
#print(v2)
for i in range(len(v[0])):
    for j in range(len(v)):
        if abs(v[j,i])>1e-12:
            print(i,'{0:b}'.format(j))
        

0 101011
0 11010100
1 101111
1 11010000
2 101011
2 11010100


задаем стартовый гамильтониан hb - тензорное произведение $\sigma^x$

In [103]:
hb = lkron([sx for i in range(size)])
#print(hb)

по
https://en.wikipedia.org/wiki/Bauer%E2%80%93Fike_theorem#The_Bauer%E2%80%93Fike_Theorem
будем определять на какой шаг можно прыгнуть

эволюцию начнем с hp и будем двигаться к hb

In [137]:
dparam = abs(sp.linalg.eigsh(-hp+hb,1,which='LM',return_eigenvectors =False)[0])
dparam

0.04296874999999986

In [716]:
param = 0 # 0 - hp, 1 - hb
e = sorted(sp.linalg.eigsh((1-param)*hp+param*hb,3,which='SA',return_eigenvectors =False))
print(param,e)
while e[2]-e[1]>1e-15:
    param+=0.9*(e[2]-e[1])/dparam
    e = sorted(sp.linalg.eigsh((1-param)*hp+param*hb,3,which='SA',return_eigenvectors =False))
    print(param,e)

0 [4.4872713469857415e-19, 3.555244990507793e-18, 0.0039062499999999996]
0.081818181818182 [-0.0003196022727272701, 0.0003196022727272747, 0.0032670454545454717]
0.14355371900826516 [-0.0005607567148760382, 0.0005607567148760381, 0.0027847365702479304]
0.19013598797896386 [-0.000742718703042825, 0.0007427187030428297, 0.002420812593914353]
0.22528442729321843 [-0.000880017294114137, 0.0008800172941141317, 0.002146215411771739]
0.25180552241215604 [-0.0009836153219224804, 0.0009836153219224886, 0.0019390193561550349]
0.2718168941837178 [-0.0010617847429051455, 0.0010617847429051502, 0.001782680514189707]
0.2869163837931689 [-0.0011207671241920598, 0.0011207671241920713, 0.0016647157516158752]
0.29830963504393665 [-0.0011652720118903762, 0.0011652720118903773, 0.0015757059762192445]
0.3069063609876977 [-0.0011988529726081971, 0.001198852972608195, 0.0015085440547836213]
0.31339298147253575 [-0.0012241913338770908, 0.0012241913338770982, 0.0014578673322458205]
0.31828743147473154 [-0.0012

0.3333333333320186 [-0.001302083333328198, 0.0013020833333282009, 0.0013020833333436074]
0.3333333333323413 [-0.0013020833333294564, 0.0013020833333294607, 0.0013020833333410894]
0.33333333333258486 [-0.0013020833333304105, 0.0013020833333304118, 0.0013020833333391825]
0.33333333333276854 [-0.0013020833333311243, 0.001302083333331132, 0.0013020833333377486]
0.33333333333290716 [-0.0013020833333316662, 0.0013020833333316707, 0.0013020833333366667]
0.3333333333330118 [-0.0013020833333320736, 0.0013020833333320795, 0.0013020833333358451]
0.3333333333330907 [-0.0013020833333323837, 0.001302083333332388, 0.001302083333335229]
0.3333333333331502 [-0.001302083333332619, 0.0013020833333326123, 0.0013020833333347629]
0.3333333333331952 [-0.0013020833333327948, 0.001302083333333548, 0.0013020833333336642]


в первом столбце param, и дальше 3 СЗ (первое - ground state)

и видно, что, приближаясь к param=0.(3), 2е и 3е СЗ сливаются